# Job Application Client Notebook

This notebook provides an interactive interface for running the job application process using the CrewAI framework.

In [ ]:
import sys
import os

# Add the parent directory to sys.path to be able to import the crew module
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from crewai import Crew
from crew.agents import create_agents
from crew.tasks import create_tasks
from crew.utils import load_config, print_llm_assignments

## Configuration

Set the paths for your resume and configuration file.

In [ ]:
resume_path = "path/to/your/resume.md"
config_path = "path/to/your/config.json"

# Load configuration
config = load_config(config_path)

# Check for Serper API key
if 'serper' not in config['api_keys'] or not config['api_keys']['serper']:
    raise ValueError("Serper API key is missing in the config file. Please add it to continue.")

# Print LLM assignments
print_llm_assignments(config)

## Job Information

Enter the job posting URL and optionally, the job description.

In [ ]:
job_posting_url = input("Enter the job posting URL: ")
job_description = input("Enter the job description (optional, press Enter to skip): ")

# Add these to the config
config['job_posting_url'] = job_posting_url
config['job_description'] = job_description

## Create Agents and Tasks

In [ ]:
# Create agents
job_analyzer, relevance_selector, emphasis_strategist, cover_letter_writer = create_agents(resume_path, config)

# Create tasks
job_analysis_task, relevance_task, emphasis_task, cover_letter_task = create_tasks(
    job_analyzer, relevance_selector, emphasis_strategist, cover_letter_writer
)

## Create and Run the Crew

In [ ]:
def run_job_application_process(resume_path, config):
    # Create and run the crew
    job_application_crew = Crew(
        agents=[job_analyzer, relevance_selector, emphasis_strategist, cover_letter_writer],
        tasks=[job_analysis_task, relevance_task, emphasis_task, cover_letter_task],
        verbose=True
    )

    # Prepare inputs for the crew
    job_application_inputs = {
        'job_posting_url': config['job_posting_url'],
        'super_resume_path': resume_path,
        'job_description': config['job_description']
    }

    # Run the crew
    result = job_application_crew.kickoff(inputs=job_application_inputs)
    return result

print("Starting job application process...")
result = run_job_application_process(resume_path, config)

print("\nJob Application Process Completed")
print("\nResults:")
print(result)

## Review Output Files

The process should have generated several output files. Let's review them:

In [ ]:
output_files = ['focused_resume.md', 'emphasize_strategy.md', 'cover_letter.md']

for file in output_files:
    print(f"\nContents of {file}:")
    print("-" * 40)
    try:
        with open(file, 'r') as f:
            print(f.read())
    except FileNotFoundError:
        print(f"File {file} not found. It may not have been generated.")

## Next Steps

1. Review the generated files and make any necessary adjustments.
2. Use the focused resume and tailored cover letter for your job application.
3. Consider the emphasis strategy when formatting your resume.

Good luck with your job application!